<center> <h1 style="background-color:DarkSlateBlue; color:white" >Movie Recommendation System</h1> 

![Image](https://www.vshsolutions.com/wp-content/uploads/2020/02/recommender-system-for-movie-recommendation.jpg)

<center>
<br>    
<a id="top"></a>    
<div class="list-group" id="list-tab" role="tablist">
  <h3 class="list-group-item list-group-item-action active" style="background-color:DarkSlateBlue; color:white" data-toggle="list"  role="tab" aria-controls="home">Notebook Content!</h3>  
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#Required libraries" role="tab" aria-controls="profile" style="color:DarkSlateBlue">Required libraries<span class="badge badge-primary badge-pill" style="background-color:steelblue; color:white">1</span></a>
   <a class="list-group-item list-group-item-action" data-toggle="list" href="#I/O" role="tab" aria-controls="profile" style="color:DarkSlateBlue">I/O<span class="badge badge-primary badge-pill" style="background-color:steelblue; color:white">2</span></a>
    <a class="list-group-item list-group-item-action" data-toggle="list" href="#Custom functions" role="tab" aria-controls="profile" style="color:DarkSlateBlue">Custom functions<span class="badge badge-primary badge-pill" style="background-color:steelblue; color:white">3</span></a>
    <a class="list-group-item list-group-item-action" data-toggle="list" href="#Data loading" role="tab" aria-controls="profile" style="color:DarkSlateBlue">Data loading<span class="badge badge-primary badge-pill" style="background-color:steelblue; color:white">4</span></a>
    <a class="list-group-item list-group-item-action" data-toggle="list" href="#Collaborative Filtering - User Based" role="tab" aria-controls="profile" style="color:DarkSlateBlue">Collaborative Filtering - User Based<span class="badge badge-primary badge-pill" style="background-color:steelblue; color:white">5</span></a>
    <a class="list-group-item list-group-item-action" data-toggle="list" href="#Collaborative Filtering - Item Based" role="tab" aria-controls="profile" style="color:DarkSlateBlue">Collaborative Filtering - Item Based<span class="badge badge-primary badge-pill" style="background-color:steelblue; color:white">6</span></a>

<a id='Required libraries'></a>
<h1 style="color:DarkSlateBlue" >Required libraries</h1> 

<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover">Go to TOC</a>

In [1]:
import sys, os
import re
path_entorno = sys.executable
if "env" in path_entorno:
    regexp = re.compile("envs(.*)$")
    entorno = regexp.search(path_entorno).group(1)
    print("Entorno:",entorno)
else:
    print("Entorno: base")
directorio = os.getcwd()
print("Directorio:",directorio)

Entorno: \env_recommender\python.exe
Directorio: C:\Users\fc00242\Desktop\Proyectos_Python\Mios\Github\Recommender Using Collaborative Filtering


In [2]:
#################Libraries##############
#General libraries
import os

#Data analysis libraries
import pandas as pd
pd.options.display.max_colwidth = 1000
import numpy as np
import operator
import ast
pd.set_option("display.max_columns",100)
pd.set_option("display.max_rows",100)

#Visualization libraries
from matplotlib import pyplot as plt
from wordcloud import WordCloud
import plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
plotly.offline.init_notebook_mode (connected = True)
import ipywidgets as widgets

#Sklearn
from mlxtend.preprocessing import TransactionEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel,cosine_similarity

#Nltk
import nltk
#nltk.download('wordnet')
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

#Image processing
from PIL import Image
import requests
from io import BytesIO

#String similarity
from pyjarowinkler import distance as jaro_distance
from strsimpy.normalized_levenshtein import NormalizedLevenshtein
from strsimpy.jaro_winkler import JaroWinkler

#Singular value decomposition
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

############################WARNINGS######################
import warnings
warnings.filterwarnings('ignore')

##################################DISPLAY###################################
from IPython.core.display import display, HTML,clear_output, Markdown
display(HTML(
    '<style>'
        '#notebook { padding-top:0px !important; } ' 
        '.container { width:90% !important; } '
        '.end_space { min-height:0px !important; } '
    '</style>'
))

#from itertools import combinations,product
#!pip install strsimpy
#from strsimpy.normalized_levenshtein import NormalizedLevenshtein

<a id='I/O'></a>
<h1 style="color:DarkSlateBlue" >I/O</h1> 

<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover">Go to TOC</a>

In [3]:
path_metadata = './dat/movies_metadata.csv'
path_links_small = './dat/links_small.csv'
path_ratings_small = './dat/ratings_small.csv'

<a id='Custom functions'></a>
<h1 style="color:DarkSlateBlue" >Custom functions</h1> 

<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover">Go to TOC</a>

In [4]:
def calculate_jaro_distance(*, selected_title, all_possible_titles, num_similarities):
    #Jaro similarity
    similarity_jaro_list = []
    title_names = [title for title in all_possible_titles if str(title)!="nan"]
    for title_name in title_names:
        similarity_jaro = jaro_distance.get_jaro_distance(selected_title.lower(), title_name.lower())
        similarity_jaro_list.append(similarity_jaro)
    titles_dict_jaro = dict(zip(title_names, similarity_jaro_list))
    titles_dict_sorted_jaro = dict(sorted(titles_dict_jaro.items(), key=operator.itemgetter(1),reverse=True)[:num_similarities])
    df_similarities_jaro = pd.DataFrame(list(titles_dict_sorted_jaro.items()),columns = ['title','similarity']) 
    return df_similarities_jaro

In [5]:
def get_most_likely_items_cosine_similarity(*,items,max_number_of_predictions,df_similarity):
    df_transactions_cosine = df_similarity[df_similarity['title'].isin(items)].drop(columns=items)
    display(df_transactions_cosine.head())
    df_most_similar_items = df_transactions_cosine.drop(columns=['title']).sum(axis = 0).reset_index().rename(columns={0:'similarity'}).sort_values(by="similarity",ascending=False)
    fig = px.bar(df_most_similar_items.head(max_number_of_predictions), x="title",y="similarity",title="Recommended movies (using cosine similarities)", 
           labels={'producto': "Most likely product to buy"}, height=500)
    fig.show()

<a id='Data loading'></a>
<h1 style="color:DarkSlateBlue" >Data loading</h1> 

<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover">Go to TOC</a>

In [6]:
#################  1) METADATA DATASET     ##########################
df_metadata = pd.read_csv(path_metadata,low_memory=False)
df_metadata['id'] = df_metadata['id'].astype('str')
df_metadata['title'] = df_metadata['title'].str.title()
df_metadata['revenue'] = df_metadata['revenue'].astype('float')
df_metadata['imdb_id'] = df_metadata['imdb_id'].str.replace('tt','')

################   2) LINKS DATASET         #####################
df_links = pd.read_csv(path_links_small,low_memory=False,dtype=str)
display(df_links.head())

################   3) RATINGS SMALL DATASET ####################
df_ratings = pd.read_csv(path_ratings_small,dtype=str)
display(df_ratings.head())


#Merge three dataframes
df_ratings = df_ratings.\
            merge(df_links,how="inner").\
            merge(df_metadata[['imdb_id','title']].drop_duplicates('title'),how="inner",left_on="imdbId",right_on="imdb_id")

df_ratings = df_ratings[['userId','title','rating']]
df_ratings['userId'] = df_ratings['userId'].astype(str)
df_ratings['rating'] = df_ratings['rating'].astype(float)
display(df_ratings.head())


#Some information about our data
number_of_users = df_ratings[['userId']].drop_duplicates().shape[0]
number_of_movies = df_ratings[['title']].drop_duplicates().shape[0]
number_of_ratings = df_ratings.shape[0]
number_of_ratings_per_user = round(df_ratings.shape[0]/number_of_users)
print("There are {} different users, {} different movies and {} ratings in total.".format(number_of_users,number_of_movies,number_of_ratings))
print("On average every user has rated {} movies".format(number_of_ratings_per_user))

,movieId,imdbId,tmdbId
0,1,0114709,862
1,2,0113497,8844
2,3,0113228,15602
3,4,0114885,31357
4,5,0113041,11862


,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


,userId,title,rating
0,1,Dangerous Minds,2.5
1,7,Dangerous Minds,3.0
2,31,Dangerous Minds,4.0
3,32,Dangerous Minds,4.0
4,36,Dangerous Minds,3.0


There are 671 different users, 8623 different movies and 97634 ratings in total.
On average every user has rated 146 movies


In [7]:
#Image that we will use in the notebook
response = requests.get("https://wpamelia.com/wp-content/uploads/2019/06/loading1.jpg")
image = Image.open(BytesIO(response.content)) 

<a id='Collaborative Filtering - User Based'></a>
<h1 style="color:DarkSlateBlue"> Collaborative Filtering - User Based </h1> 

<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover">Go to TOC</a>

In [8]:
#Let's make the ratings dataset a surprise dataset
reader = Reader()
data = Dataset.load_from_df(df_ratings[['userId', 'title', 'rating']], reader)

# Run 5-fold cross-validation and then print results
svd = SVD(random_state=111)
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

#Let's train our model
trainset = data.build_full_trainset()
svd.fit(trainset)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8959  0.9055  0.8950  0.8965  0.8887  0.8963  0.0054  
MAE (testset)     0.6915  0.6956  0.6870  0.6903  0.6844  0.6898  0.0038  
Fit time          4.42    4.37    4.57    4.53    4.25    4.43    0.11    
Test time         0.17    0.11    0.11    0.11    0.11    0.12    0.03    


In [9]:
############################RATINGS############################
fig = px.histogram(df_ratings,x="rating",title="Distribution of ratings")
fig.update_layout(autosize=False,width=800,height=400).show()


#############################USERS#############################    
#Users that have rated many movies
active_users = df_ratings.groupby("userId").agg({'rating':'count'}).reset_index().\
                rename(columns={'rating':'number_of_ratings'}).sort_values(by="number_of_ratings",ascending=False)
active_users['userId'] = 'ID' + active_users['userId']

#Figure                                                                          
fig = px.bar(active_users.head(20),
            x="userId",y="number_of_ratings",
            title="Users with highest number of ratings")
fig.update_layout(autosize=False,width=800,height=400).show()

                                                                           
##############################MOVIES###############################
#Popular movies
popular_movies = df_ratings.groupby("title").agg({'userId':'count','rating':'mean'}).reset_index().\
                rename(columns={'userId':'number_of_ratings','rating':'average_rating'}).sort_values(by="number_of_ratings",ascending=False)
                                                                           
#Figure
fig = px.bar(popular_movies.head(20),
             x="title",y="number_of_ratings",#color="title",
             title="Movies with highest number of ratings")
fig.update_layout(autosize=False,width=800,height=400).show()

Our main goal would be to predict what are the top recommendations for different user according to their personal taste.

In [10]:
#Let's predict
@widgets.interact
def get_recommendations(user = '1'):
    users = df_ratings['userId'].unique()
    if user in users:
        #Barchart of favourite movies by user
        movies_rated_by_user = df_ratings[df_ratings['userId'] == user].sort_values(by=["userId","rating"],ascending=False)
        movies_list = movies_rated_by_user['title'].tolist()
        #print("Number of ratings by user {}:".format(user),movies_rated_by_user.shape[0], "// Average rating by user {}:".format(user),movies_rated_by_user['rating'].mean())
        top_movies_rated_by_user = movies_rated_by_user[movies_rated_by_user['rating']>=3]
        fig1 = px.bar(top_movies_rated_by_user.head(10),x="title",y="rating",
                       color="title",
                       color_discrete_sequence = px.colors.qualitative.Vivid,
                       title="Highest rated movies by user {}".format(user))
        #fig1.update_layout(autosize=False,width=800,height=350).show()

        #Recommendations
        title_names = df_ratings['title'].unique()
        predicted_ratings = []
        for title in title_names:
            predicted_rating = svd.predict(user, title)
            predicted_ratings.append(predicted_rating.est)
        titles_dict = dict(zip(title_names, predicted_ratings))
        titles_dict_sorted = dict(sorted(titles_dict.items(), key=operator.itemgetter(1),reverse=True))
        df_recommended_movies = pd.DataFrame(list(titles_dict_sorted.items()),columns = ['title','predicted_ratings'])
        
        df_recommended_movies = df_recommended_movies[~df_recommended_movies['title'].isin(movies_list)].head(10)

        #Barchart of the recommendations
        fig2 = px.bar(df_recommended_movies,x="title",y="predicted_ratings",
                       color="title",
                       color_discrete_sequence = px.colors.qualitative.Dark24,
                       title="Recommended movies for user {}".format(user))
        #fig2.update_layout(autosize=False,width=800,height=400).show()


        #Figure
        fig = make_subplots(rows=1, cols=2,subplot_titles=("Highest rated movies by user {} - movies already watched".format(user),
                                                           "Recommended movies for user {}".format(user)))


        for trace in fig1.data:
            fig.add_trace(trace, 1, 1)
        for trace in fig2.data:
            fig.add_trace(trace, 1, 2)

        fig.update_layout(barmode="stack")
        fig.update_layout(autosize=False,width=1400,height=600,showlegend=False).show()
    else:
        display(image)

interactive(children=(Text(value='1', description='user'), Output()), _dom_classes=('widget-interact',))

There is a **big problem** with all the **user based techniques**. If we want to recommend a movie to a new user, we can't do it without retraining the model with the new user which is really computationally expensive. That's why we will see how to implement an item based collaborative filtering, where we can have an item similarity matrix pre-computed, and we just need to recommend movies that are similar to the ones the user has liked.

<a id='Collaborative Filtering - Item Based'></a>
<h1 style="color:DarkSlateBlue"> Collaborative Filtering - Item Based </h1> 

<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover">Go to TOC</a>

In [14]:
# Dataframe where every row is an user, the columns are all the possible movies and the values are the ratings (NaN if the user has not watched the movie)
pivot_ratings = df_ratings.pivot(index='userId', columns='title', values='rating').fillna(0) #pivot_ratings = pd.pivot_table(df_ratings,index='userId', columns='title', values='rating')
display(pivot_ratings.head())

# Cosine similarity matrix
similarity = cosine_similarity(pivot_ratings.T)##The distance is a positive measure, and we are calculating the similarity as 1-distance
df_similarity = pd.DataFrame(similarity, index=pivot_ratings.columns, columns=pivot_ratings.columns).reset_index()

title,$9.99,'Neath The Arizona Skies,"'Night, Mother",(500) Days Of Summer,...And God Created Woman,...And Justice For All,1-900,10,10 Attitudes,10 Cloverfield Lane,10 Items Or Less,10 Things I Hate About You,10 Years,"10,000 Bc",100 Girls,100 Rifles,101 Dalmatians,101 Reykjavik,102 Dalmatians,11'09''01 - September 11,11:14,12 Angry Men,12 Years A Slave,127 Hours,12:08 East Of Bucharest,13 Assassins,13 Ghosts,13 Going On 30,13 Tzameti,1408,15 Minutes,16 Blocks,16 Wishes,17 Again,18 Again!,1941,1984,1990: The Bronx Warriors,2 Days In New York,2 Days In The Valley,2 Fast 2 Furious,2 Guns,2 Or 3 Things I Know About Her,20 Dates,20 Feet From Stardom,"20,000 Leagues Under The Sea",200 Cigarettes,200 Motels,2001: A Space Odyssey,2010,...,Young Guns,Young Guns Ii,Young People Fucking,Young Sherlock Holmes,Youngblood,Your Friends & Neighbors,Your Highness,Your Sister'S Sister,"Yours, Mine And Ours",Youth In Revolt,Youth Of The Beast,Yu-Gi-Oh! The Movie,Z,Z Channel: A Magnificent Obsession,Zach Galifianakis: Live At The Purple Onion,Zack And Miri Make A Porno,Zardoz,Zathura: A Space Adventure,Zatoichi,Zatôichi On The Road,Zazie Dans Le Métro,Zeitgeist,Zelig,Zenon: Girl Of The 21St Century,Zenon: The Zequel,Zenon: Z3,Zero Dark Thirty,Zero Effect,Zerophillia,Zodiac,Zombeavers,Zombie Flesh Eaters,Zombie Holocaust,Zombieland,Zoolander,Zoolander 2,Zoom,Zoot Suit,Zootopia,Zorba The Greek,Zorns Lemma,"Zorro, The Gay Blade",Zulu,[Rec],¡Three Amigos!,À Nous La Liberté,Æon Flux,İtirazım Var,Želary,’Round Midnight
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
102,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
df_similarity

title,title,$9.99,'Neath The Arizona Skies,"'Night, Mother",(500) Days Of Summer,...And God Created Woman,...And Justice For All,1-900,10,10 Attitudes,10 Cloverfield Lane,10 Items Or Less,10 Things I Hate About You,10 Years,"10,000 Bc",100 Girls,100 Rifles,101 Dalmatians,101 Reykjavik,102 Dalmatians,11'09''01 - September 11,11:14,12 Angry Men,12 Years A Slave,127 Hours,12:08 East Of Bucharest,13 Assassins,13 Ghosts,13 Going On 30,13 Tzameti,1408,15 Minutes,16 Blocks,16 Wishes,17 Again,18 Again!,1941,1984,1990: The Bronx Warriors,2 Days In New York,2 Days In The Valley,2 Fast 2 Furious,2 Guns,2 Or 3 Things I Know About Her,20 Dates,20 Feet From Stardom,"20,000 Leagues Under The Sea",200 Cigarettes,200 Motels,2001: A Space Odyssey,...,Young Guns,Young Guns Ii,Young People Fucking,Young Sherlock Holmes,Youngblood,Your Friends & Neighbors,Your Highness,Your Sister'S Sister,"Yours, Mine And Ours",Youth In Revolt,Youth Of The Beast,Yu-Gi-Oh! The Movie,Z,Z Channel: A Magnificent Obsession,Zach Galifianakis: Live At The Purple Onion,Zack And Miri Make A Porno,Zardoz,Zathura: A Space Adventure,Zatoichi,Zatôichi On The Road,Zazie Dans Le Métro,Zeitgeist,Zelig,Zenon: Girl Of The 21St Century,Zenon: The Zequel,Zenon: Z3,Zero Dark Thirty,Zero Effect,Zerophillia,Zodiac,Zombeavers,Zombie Flesh Eaters,Zombie Holocaust,Zombieland,Zoolander,Zoolander 2,Zoom,Zoot Suit,Zootopia,Zorba The Greek,Zorns Lemma,"Zorro, The Gay Blade",Zulu,[Rec],¡Three Amigos!,À Nous La Liberté,Æon Flux,İtirazım Var,Želary,’Round Midnight
0,$9.99,1.00000,0.000000,0.0,0.156330,0.0,0.077954,0.0,0.000000,0.0,0.000000,0.000000,0.026649,0.000000,0.000000,0.0,0.000000,0.028796,0.0,0.000000,0.00000,0.000000,0.039008,0.000000,0.061295,0.365636,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.048860,0.000000,0.0,0.000000,0.000000,0.040037,0.000000,0.000000,0.036691,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.162199,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.145133,0.0,0.0,0.0,0.000000,0.000000,0.0,0.071848,0.000000,0.000000,0.0,0.047953,0.031535,0.0,0.000000,0.0,0.000000,0.0,0.365636,0.0,0.0,0.000000,0.058218,0.0,0.000000,0.000000,0.0,0.000000
1,'Neath The Arizona Skies,0.00000,1.000000,0.0,0.019259,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.018221,0.000000,0.105409,0.0,1.000000,0.026252,0.0,0.072932,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.124035
2,"'Night, Mother",0.00000,0.000000,1.0,0.000000,0.0,0.369274,0.0,0.000000,0.0,0.000000,0.000000,0.084159,0.000000,0.000000,0.0,0.000000,0.242508,0.0,0.000000,0.00000,0.000000,0.076994,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.166558,0.000000,0.000000,0.0,0.161296,0.000000,0.227593,0.668511,0.000000,0.141620,...,0.160748,0.203135,0.000000,0.169842,0.0,0.270073,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.317034,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.214498,0.0,0.000000,0.000000,0.0,0.000000
3,(500) Days Of Summer,0.15633,0.019259,0.0,1.000000,0.0,0.058854,0.0,0.113989,0.0,0.196231,0.192593,0.283897,0.096296,0.121806,0.0,0.019259,0.129939,0.0,0.094110,0.00000,0.115953,0.22

In [12]:
#Let's plot the results
# Button, text box, output
layout = widgets.Layout(width='400px', height='25px') #set width and height
butt_collaborative_filtering = widgets.Button(description='Display similar movies based on collaborative filtering',layout = layout,button_style='success')
items_collaborative_filtering = widgets.Text(value = 'The Dark Knight',description='Titles',layout=layout)
num_similar_items_collaborative_filtering = widgets.Dropdown(options=list(range(1,21)),value=10,description='Num similar items',disabled=False,layout=layout)
output_collaborative_filtering = widgets.Output()

def on_butt_clicked(b):
    with output_collaborative_filtering:
        clear_output()
        itemset = list(items_collaborative_filtering.value.split(","))
        itemset = [item.strip().title() for item in itemset]
        #print(itemset)
        items_no_reconocidos = [item for item in itemset if item not in list(df_ratings['title'].unique())]
        if len(items_no_reconocidos)==0:
            #################################ITEM-TO-ITEM COLLABORATIVE FILTERING USING COSINE SIMILARITIES##########################
            get_most_likely_items_cosine_similarity(items = itemset, max_number_of_predictions = num_similar_items_collaborative_filtering.value, df_similarity = df_similarity)
        else:
            if len(items_no_reconocidos)==1:
                print(f"The title {items_no_reconocidos} is not recognised", "\n")
            if len(items_no_reconocidos)>1:
                print(f"The titles {items_no_reconocidos} are not recognised", "\n")
            for item in items_no_reconocidos:
                df_similarities_jaro = calculate_jaro_distance(selected_title = item, all_possible_titles = df_ratings['title'].unique(), num_similarities = 1)
                print("Maybe you meant:",df_similarities_jaro['title'].values[0])
            
        
butt_collaborative_filtering.on_click(on_butt_clicked)
widgets.VBox([butt_collaborative_filtering,
              items_collaborative_filtering,
              num_similar_items_collaborative_filtering,
              output_collaborative_filtering])  